# 数据预处理

## 数据加载

In [10]:
import json
with open('D:/model/web/nlp03/data/train.json', 'r', encoding='utf-8') as f:
    txt = json.load(f)

In [11]:
len(txt)

5881

In [12]:
# 数据整理
res = {'context': [], 'Q': [], 'A': []}
for sample in txt:
    for i in sample['annotations']:
        t = sample['text']
        res['Q'].append(i['Q'])
        res['A'].append(i['A'])
        res['context'].append(t if '"' != t[0] else t[1:])

In [13]:
import pandas as pd 

In [15]:
total_data = pd.DataFrame(res)
total_data.head()

,context,Q,A
0,胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习...,什么类型的胆囊结石可不作治疗？,无症状胆囊结
1,胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习...,胆石症的治疗应注意什么？,应区别不同情况分别处理
2,胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习...,胆管结石宜采用什么样的治疗方式？,以手术为主的综合治疗
3,反佐配伍的典范，始见于张仲景《伤寒杂病论》，其中记载“干呕、吐涎沫、头痛者吴茱萸汤主之”。患...,“干呕、吐涎沫、头痛者吴茱萸汤主之”这句话曾出现在哪本医学巨著中？,《伤寒杂病论》
4,反佐配伍的典范，始见于张仲景《伤寒杂病论》，其中记载“干呕、吐涎沫、头痛者吴茱萸汤主之”。患...,《伤寒杂病论》的作者是谁？,张仲景


In [16]:
total_data.shape

(18478, 3)

In [17]:
# 检查并确认所有的答案A在原文context中出现过
# 只取答案A出现在原文context中的例子
ind = total_data.apply(lambda x: x['A'] in x['context'], axis=1)
total_data = total_data.loc[ind, :]
print(total_data.shape)

(18456, 3)


In [18]:
# 确认答案在上下文的位置信息（开始、结束）
total_data['start'] = total_data.apply(lambda x: x['context'].find(x['A']), axis=1)
total_data['end'] = total_data['start'] + total_data['A'].apply(len)

In [19]:
total_data

,context,Q,A,start,end
0,胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习...,什么类型的胆囊结石可不作治疗？,无症状胆囊结,18,24
1,胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习...,胆石症的治疗应注意什么？,应区别不同情况分别处理,6,17
2,胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习...,胆管结石宜采用什么样的治疗方式？,以手术为主的综合治疗,94,104
3,反佐配伍的典范，始见于张仲景《伤寒杂病论》，其中记载“干呕、吐涎沫、头痛者吴茱萸汤主之”。患...,“干呕、吐涎沫、头痛者吴茱萸汤主之”这句话曾出现在哪本医学巨著中？,《伤寒杂病论》,14,21
4,反佐配伍的典范，始见于张仲景《伤寒杂病论》，其中记载“干呕、吐涎沫、头痛者吴茱萸汤主之”。患...,《伤寒杂病论》的作者是谁？,张仲景,11,14
...,...,...,...,...,...
18473,痘痘，医学上叫痤疮，是毛囊皮脂腺的慢性炎症。导致痘痘发生有三个环节：一是局部皮脂分泌过多，这...,爱长痘的人饮食需要注意哪些？,爱长痘、出油多的人应少摄入油腻食物、甜食等高热量的食品，而多吃水果、蔬菜及粗纤维食物。再次，...,356,460
18474,俗话说“树落叶，人落发”，脱发最容易在秋季发生和加重。外用洗护方配合正确的梳头方式，可有效减...,如何减少脱发？,外用洗护方配合正确的梳头方式，可有效减少脱发,27,49
18475,俗话说“树落叶，人落发”，脱发最容易在秋季发生和加重。外用洗护方配合正确的梳头方式，可有效减...,生姜的功效是什么？,生姜性温，具有解表、发散的功效,50,65
18476,俗话说“树落叶，人落发”，脱发最容易在秋季发生和加重。外用洗护方配合正确的梳头方式，可有效减...,姜汁防脱发的外用方法有哪几种？,一是把生姜切成片放在少量水中熬煮，也可加入枸杞一起熬煮，煮到汤汁变黄，有浓烈姜味溢出即可。把...,116,353


## 数据划分

In [21]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(total_data, random_state=42, test_size=0.2)
print(train.shape, test.shape)

(14764, 5) (3692, 5)


## 数据准备

In [33]:
from transformers import AutoTokenizer
model_path = 'D:/model/web/nlp03/roberta-base-chinese-extractive-qa'
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [34]:
tokenizer 

BertTokenizerFast(name_or_path='D:/model/web/nlp03/roberta-base-chinese-extractive-qa', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [35]:
import numpy as np
def prepare_train_features(examples):
    tokenizer_examples = tokenizer(
        list(examples['Q']), 
        list(examples['context']),
        max_length=512,   # 最大长度
        padding='max_length',  # 不足：填充
        truncation='only_second',  # 超过：截取（只截取上下文，问题不处理）
        stride=128,  # 重叠部分长度
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
    )
    # 由于一个示例可能会为我们提供多个特征，如果它具有很长的上下文，那么我们需要一个从特征到其相应样本的映射
    sample_mapping = tokenizer_examples.pop('overflow_to_sample_mapping')
    # offset_mapping将为我们提供从标记到原始上下文中字符位置的映射。这将帮助我们计算start位置和end位置
    offset_mapping = tokenizer_examples.pop('offset_mapping')
    
    tokenizer_examples['start_positions'] = []
    tokenizer_examples['end_positions'] = []

    for i, offset in enumerate(offset_mapping):

        start_char, end_char = list(examples['start'])[sample_mapping[i]], list(examples['end'])[sample_mapping[i]]

        # tokenizer_examples['token_type_ids']
        t = np.where(np.array(tokenizer_examples.sequence_ids(i)) == 1)[0]
        context_start, context_end = min(t), max(t)

        # 判断答案是否存在在截断的上下文中，如果否，则标记为（0,0）；如果是，则更新开始、结束的位置信息
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            # 判断答案是否完整存在在截断的上下文中
            tokenizer_examples['start_positions'].append(0)
            tokenizer_examples['end_positions'].append(0)
        else:
            tokenizer_examples['start_positions'].append(
                [k for k,n in enumerate(offset) if k >= context_start and start_char in range(n[0], n[1])][0]
            )
            tokenizer_examples['end_positions'].append(
                [k+1 for k,n in enumerate(offset) if k >= context_start and end_char in range(n[0], n[1]+1)][0]
            )
    return tokenizer_examples

In [36]:
train_token = prepare_train_features(train)
test_token = prepare_train_features(test)

In [37]:
from datasets import Dataset
train_ds = Dataset.from_dict(train_token)
test_ds = Dataset.from_dict(test_token)

In [38]:
train_ds

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 15547
})

# 模型训练

In [39]:
# 模型加载
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

In [40]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [54]:
#定义模型参数
from transformers import TrainingArguments
model_ckpt_dir = 'D:/model/web/nlp03/test-qa'
batch_size = 8
args = TrainingArguments(
    model_ckpt_dir,
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size, 
    per_gpu_train_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=3,
)

In [55]:
# 定义数据校对器
from transformers import default_data_collator
data_collator = default_data_collator

In [56]:
# 自定义评估指标计算函数
from transformers.data.metrics.squad_metrics import compute_exact, compute_f1

def compute_metrics(eval_pred):
    predictions, label_ids  = eval_pred
    
    answer_start = np.argmax(predictions[0], axis=1)
    answer_end = np.argmax(predictions[1], axis=1)
    
    label_start, label_end = label_ids[0], label_ids[1]
    
    data = test_ds['input_ids']
    
    gold_answers = []
    pred_answers = []
    for idx, example in enumerate(data):
        gold_answer = tokenizer.decode(example[label_start[idx]: label_end[idx]])
        pred_answer = tokenizer.decode(example[answer_start[idx]: answer_end[idx]])
        
        gold_answers.append(gold_answer.strip())
        pred_answers.append(pred_answer.strip())
        
    f1_score, exact_score = 0, 0
    for gold_answer, pred_answer in zip(gold_answers, pred_answers):
        exact_score += compute_exact(gold_answer, pred_answer)
        f1_score += compute_f1(gold_answer, pred_answer)
    exact_score /= len(gold_answers)
    f1_score /= len(gold_answers)
    
    exact_score *= 100
    f1_score *= 100
    result = {'exact_score': exact_score, 'f1_score': f1_score}
    return result

In [57]:
# 定义trainer对象
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=train_ds, 
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [58]:
trainer.train()
# epoch=10在4060跑预计时间2小时，数据量太大，下面采用跑好的模型

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Epoch,Training Loss,Validation Loss,Exact Score,F1 Score
1,1.342900,1.261639,42.586341,77.367336


TrainOutput(global_step=1944, training_loss=1.4039190786856193, metrics={'train_runtime': 802.4812, 'train_samples_per_second': 19.374, 'train_steps_per_second': 2.422, 'total_flos': 4062380676974592.0, 'train_loss': 1.4039190786856193, 'epoch': 1.0})

# 模型评估

In [59]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
eval_dir = 'D:/model/web/nlp03/test-qa/checkpoint-77500/'
tokenizer_test = AutoTokenizer.from_pretrained(eval_dir)
model_test = AutoModelForQuestionAnswering.from_pretrained(eval_dir)

In [60]:
test_sample = total_data.iloc[0,:]
test_sample

context    胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习...
Q                                            什么类型的胆囊结石可不作治疗？
A                                                     无症状胆囊结
start                                                     18
end                                                       24
Name: 0, dtype: object

In [61]:
import torch
QA_input = tokenizer_test(
    test_sample['Q'], 
    test_sample['context'],
    max_length=512,
    padding='max_length',
    truncation='only_second',
    stride=128,
    return_tensors='pt',
)

with torch.no_grad():
    logits = model_test(**QA_input)

In [62]:
answer_start = np.argmax(logits.start_logits.numpy(), axis=1)
answer_end = np.argmax(logits.end_logits.numpy(), axis=1)

In [64]:
print(f'Context: {test_sample["context"]}\n\nQuestion: {test_sample["Q"]}\n')
print('Predict Answer:', tokenizer_test.decode(QA_input['input_ids'][0][answer_start[0]:answer_end[0]]).replace(' ', ''))
print('\nReal Answer: ', test_sample['A'])

Context: 胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习惯。有症状的胆囊结石仍以胆囊切除术为较安全有效的疗法，此外，尚可采用体外震波碎石。胆管结石宜采用以手术为主的综合治疗。胆石症的家庭治疗可采用以下方法：\n（1）一般治疗    预防和治疗肠道寄生虫病和肠道感染，以降低胆石症的发病率。胆绞痛发作期应禁食脂肪等食物，采用高碳水化合物流质饮食；缓解期应忌食富含胆固醇的食物如脑、肝、肾、蛋黄等。\n（2）增进胆汁排泄    可选用50%硫酸镁10~15毫升，餐后口服，每日3次；胆盐每次口服0.5~1克，每日3次；去氢胆酸0.25克，每日3次，餐后服用。\n（3）消除胆绞痛    轻者可卧床休息，右上腹热敷，用硝酸甘油酯0.6毫克，每3~4小时一次，含于舌下；或阿托品0.5毫克，每3~4小时肌肉注射一次。重者应住院治疗。\n（4）排石疗法以中药治疗为主，若右上腹疼痛有间歇期，无明显发热及黄疸，苔薄白，脉弦，属气滞者，用生大黄6克、木香9克、枳壳9克、金钱草30克、川楝子9克、黄苓9克，水煎服。右上腹痛为持续性，且阵发性加剧，有明显发热及黄疸，舌红苔黄，

Question: 什么类型的胆囊结石可不作治疗？

Predict Answer: 无症状胆囊结

Real Answer:  无症状胆囊结


In [65]:
def result_output(test_sample_index):
    test_sample = total_data.iloc[test_sample_index,:]
    QA_input = tokenizer_test(
    test_sample['Q'], 
    test_sample['context'],
    max_length=512,
    padding='max_length',
    truncation='only_second',
    stride=128,
    return_tensors='pt',
    )

    with torch.no_grad():
        logits = model_test(**QA_input)
    
    answer_start = np.argmax(logits.start_logits.numpy(), axis=1)
    answer_end = np.argmax(logits.end_logits.numpy(), axis=1)
    
    print(f'Context: {test_sample["context"]}\n\nQuestion: {test_sample["Q"]}\n')
    print('Predict Answer:', tokenizer_test.decode(QA_input['input_ids'][0][answer_start[0]:answer_end[0]]).replace(' ', ''))
    print('\nReal Answer: ', test_sample['A'])

In [66]:
result_output(10)

Context: 黄帝说：什麽叫做所胜？岐伯说：春胜长夏，长夏胜冬，冬胜夏，夏胜秋，秋胜春，这就是时令根据五行规律而互相胜负的情况。同时，时令又依其五行之气的属性来分别影响各脏。黄帝说：怎样知道它们之间的相胜情况呢？岐伯说：首先要推求气候到来的时间，一般从立春开始向下推算。如果时令未到而气候先期来过，称为太过，某气太过就会侵侮其所不胜之气，欺凌其所胜之气，这就叫做气淫；时令以到而气候未到，称为不及，某气不及，则其所胜之气因缺乏制约而妄行，其所生之气因缺乏资助而困弱，其所不胜则更会加以侵迫，这就叫做气迫。所谓求其至，就是要根据时令推求气候到来的早晚，要谨慎地等候时令的变化，气候的到来是可以预期的。如果搞错了时令或违反了时令与气候相合的关系，以致于分不出五行之气当旺的时间，那麽，当邪气内扰，病及于人的时候，好的医生也不能控制了。

Question: 什么叫做太过？

Predict Answer: 如果时令未到而气候先期来过，称为太过

Real Answer:  如果时令未到而气候先期来过，称为太过


In [67]:
result_output(20)

Context: （一）灸法的作用1.温通经络、祛除寒邪可用治寒邪所致疾患。2.引导气血有升提中气或引气下行，可治中气下陷、肝阳上亢之证。3.回阳固脱、补气固本治阳气虚脱证。4.行气活血、散瘀消肿能治疗各种痛证和寒性疖肿等。（二）灸法的适应证灸法对慢性病、虚寒等证较为适合，如久泄、痰饮、水肿、痿证、痹证、腹痛、胃痛、阳痿、遗尿、疝、虚劳，崩漏、阴挺、中风脱证、外科阴疽、瘰疬、瘿瘤等。

Question: 慢性病、虚寒等证的症状表现是什么？

Predict Answer: 

Real Answer:  如久泄、痰饮、水肿、痿证、痹证、腹痛、胃痛、阳痿、遗尿、疝、虚劳，崩漏、阴挺、中风脱证、外科阴疽、瘰疬、瘿瘤等。
